## Data-driven linearization
by Domrachev Ivan, B20-Ro-01

This notebook overviews the field of linearization, shows different approaches to linearize the system and provides with some applications and examples of system linearization

## Structure


The structure is the following:
1. Motivation
2. Taylor linearization
3. Koopman theory, EDMD
4. Dual Faceted Linearization
5. Applications & Examples

## Motivation

Linear systems are:
1. Have simple structure
2. Trivial to control
3. Can be analyzed using powerful mathematical tools
4. Computationally effiicent to evaluate

On the other hand, nonlinear systems are:
1. Have a complex inner structure
2. Each nonlinear system is unique
3. Very hard to control
4. Computationally hard to control

Naturally, one might wonder: is it possible to approximate nonlinear system with linear one?

Here we would go through some existing methods of doing so.

## Taylor linearizatoin

Most popular and, ironically, the most inaccurate linearization method. It is based on the Taylor Series decomposition:
$$f(x) = f(a) + f'(a) (x-a) + \frac{f''(a)}{2} (x-a)^2 + \ldots + \frac{f^{(n)}(a)}{2} (x-a)^n + \ldots \approx f(a) + f'(a) (x-a).$$
The above approximation is generalized for the vector space as following:
$$f(\bold{x}) \approx f(\bold{a}) + \frac{\partial f}{\partial \bold{x}} (\bold{a})\bold{x}$$
 

Therefore, to find an taylor linearization, one need to: 
1. Choose a point, where linearizatoin would be
2. Calculate a jacobian of the function at this point
3. The linear function is: $f(\bold{x}) = f(\bold{a}) + \frac{\partial f}{\partial \bold{x}} (\bold{a})(\bold{x} - \bold{a}) = f(\bold{a}) + A(\bold{x} - \bold{a})$

In [6]:
from darli.models import RobotModel
import casadi as cs
import numpy as np

x = cs.SX.sym('x', 1)
v = cs.SX.sym('v', 1)
state = cs.vertcat(x, v)
f = cs.Function('f', [state], [cs.vertcat(v, -cs.sin(x))])
jacobian = cs.Function('J', [state], [cs.jacobian(f(state), state)])
A = jacobian(np.zeros(2))

f_hat = cs.Function('f', [state], [A @ state])

In [8]:
cs.jacobian(f(state), state)

SX(
[[00, 1], 
 [(-cos(x)), 00]])

The Taylor linearization is commonly used for:
1. Estimating a type of equilibrium 
2. Calculating the region of attraction using Lyapunov theory (more on that later)

However, the Taylor linearization has two key disadvantages:
1. It quickly becomes inaccurate
2. It's valid only in a neighbourhood (almost always very small one) around point of linearization

## Koopman theory, Extended Dynamic Mode Decomposition (EDMD)

In the year 1931, Koopman in his work showed that *any nonlinear system could be approximated as a linear system in infinite dimensional space.* He did it by introducing so called **Koopman operator** for a dynamical system $\dot{x} = f(x)$, defined as:
$$(K\psi)(x) = \psi(f(x))$$

and showed that if will be linear, but have infinite dimensions, then every nonlinear function could be described by it.

It took around 80 years to realise, that this theory is applicable for linearization of dynamical systems. In year 2015, Matthew O. Williams noted, that one might not use the infinite linear space, but just **one that is big enough.**

Indeed, it turns out that it's usually much easier to work with high-dimensional (say, $\mathbb{R}^{100}$) linear system, rather than with low-dimensional nonlinear one. This idea got a name of **lifting linearization.**

Let's assume that we have chosed some Koopman operator $z = \psi(x)$. Now, the task is to find such a matrix $A$ such that function
$$\hat{f}(z) = Az$$
would be the best approximation of the original system. 

![](fig/lifted_linearization.jpg)

The method that Williams suggested was to collect lots of data points, and use them for creation of least squares problem:
$$\begin{equation}
\begin{aligned}
\min_{A} \quad & \sum_{j=1}^{K}{||\psi({x}_{j+1}) - A \psi(x)_j||_2}\\
\end{aligned}
\end{equation}
$$

This approach is called Extended Dynamic Mode Decomposition (EDMD) algorithm

It's worth mentioning that there are many different ways of choosing the family of functions to put in $\psi$. Among them are:
1. Radial Basis Function $\psi(x) = ||x - x_0||_2^2\ log(||x - x_0||)$
2. Polynomials
3. Trigonometric functions
   
And so on. Usually, they should have some parameters.

To sum up, the pipeline for implementing EDMD is the following:
1. Choose the function $\psi(x)$
2. Run the simulation of the nonlinear system to obtain the data samples $(x_j, v_j)$
3. Solve the optimization problem to find the matrix $A$
4. $\hat{f}(z) = Az, z_0 = \psi(x_0)$ is the resulting dynamics

> Note: The natural question is how to deduce the value $x$ from the resulting simulation. The answer is simple: no one prevents you from including the original state as a part of liftign one, i.e.:
> $$z = \psi(x) = \begin{bmatrix} x_0 \\ x_1 \\ \vdots \\ x_n \\ ||x - x_0^1||_2^2\ log(||x - x_0^1||) \\ ||x - x_0^2||_2^2\ log(||x - x_0^2||) \\ \vdots \end{bmatrix}$$

## Dual Faceted Linearization

Another approach to the problem was suggested by H. Asada in year 2019. It was based on two main ideas:
1. Many physical systems are represented as sum of nonlinear components
2. By combining the predictions from two different linear approximations of nonlinear dynamics, one could obtain a much more accurate results

<center><img src="fig/dfl.jpg" width=600 alt="my alt text"/></center>

To use this properties, he suggested to represent the given nonlinear system $\dot{x} = f(x)$ as follows:
$$\dot{x} = A_x x + A_\eta \eta + Bu,$$
where $\eta = \eta(x)$ is some nonlinear function of the original state. With that in mind, he suggested to construct *another differential equation* like the following:
$$\dot{\eta} = H_x x + H_\eta \eta + H_u u.$$
Here, the matrices $H_{x, \eta, u}$ are initially unknown, so it's possible to find them by using the data from the nonlienar system: 